In [1]:
import secrets
from hashlib import sha256

import math
import random
import pandas as pd

from datetime import datetime, date
from zoneinfo import ZoneInfo

__ENCODING__ = 'ascii'

## Cryptographic hash function
A cryptographic hash function is designed to be a one-way function, which means that it is easy to compute the hash value of an input message, but it is computationally infeasible to determine the original message from the hash value.

### Hard part
Given hash below determin the original message for:
```
af976e491ca0f11b64ac6719ed814eb36650a0e55fc04760b4e8802e339964b7
```

For a cryptographic hash function this is HARD!

### Easy part
Compute hash value for:
```
Non-magic people, more commonly known as Muggles!
```

For a given string it's easy to compute the hash.

In [2]:
message = "Non-magic people, more commonly known as Muggles!"
mu1 = sha256(message.encode(__ENCODING__)).hexdigest()
print(mu1)

af976e491ca0f11b64ac6719ed814eb36650a0e55fc04760b4e8802e339964b7


## SHA256
The SHA functions have the following characteristics:

1. Variable input length and fixed output length: SHA (Secure Hash Algorithm) functions can accept messages of any size and produce a fixed-length output, which is typically 160, 256, 384, or 512 bits in length, depending on the specific version of SHA being used.
2. One-way function: The one-way property of SHA means that it is easy to compute the hash value of a message, but it is computationally infeasible to reconstruct the original message from the hash value. This property is also referred to as the pre-image resistance.
3. Deterministic: For a given input, the output of a SHA function is always the same. This is important for verification purposes, as it allows different parties to independently compute and compare the hash values of a message to ensure that they are identical.
4. Collision resistance: It is computationally infeasible to find two different input messages that produce the same output hash value. This is important for security purposes, as a hash function with a collision vulnerability could be exploited by an attacker to create a fraudulent message that has the same hash value as a legitimate message.
5. Avalanche effect: Even a small change in the input message results in a significant change in the output hash value. This property ensures that any modification to the message will produce a completely different hash value, which in turn makes it difficult for an attacker to modify the message without being detected.

In [3]:
# Remove comma ',' after 'people' and diff hashes
message = "Non-magic people more commonly known as Muggles!"
mu2 = sha256(message.encode(__ENCODING__)).hexdigest()
print(mu1)
print(mu2)

af976e491ca0f11b64ac6719ed814eb36650a0e55fc04760b4e8802e339964b7
86a979d45274c9c3b0ff7615eb1de0dbe5ef2a890c430a1bd7421f094a59fc84


### Preimage attack

A preimage attack on a cryptographic hash function involves finding a message that hashes to a given hash value. In other words, given a hash output, the attacker tries to find a message that produces that hash output when fed into the hash function.
A successful SHA preimage attack would allow an attacker to create a fake message with the same hash value as the original message, which could be used to perform various malicious activities.
However, it's important to note that SHA256 are believed to be resistant to preimage attacks, otherwise it would be a significant problem for cryptographic security.


## Hash chain
A cryptographic hash function takes an input message of arbitrary size and produces a fixed-size output, known as the hash value or digest.
A hash chain is then a sequence of hash values, where each value is computed by applying a cryptographic hash function to the previous value in the chain.

It is designed in such a way that it is easy to compute the hash value for any given input, but it is computationally infeasible to find two different inputs that produce the same hash value, or to recover the input message from the hash value.

```
.------.      .------.      .------.      .------.      .------.      .----------.      .--------.       
|      |      |      |      |      |      |      |      |      |      |          |      |        |       
| H(s) |  -̶>̶  |H1(d0)|  -̶>̶  |H2(d1)|  -̶>̶  |H3(d2)|  -̶>̶  | .... |  -̶>̶  |Hk-1(dk-2)|  -̶>̶  |Hk(dk-1)|  
|      |      |      |      |      |      |      |      |      |      |          |      |        |      
`------'      `------'      `------'      `------'      `------'      `----------'      `--------'      

/-------------------------------------------------------------------------------------------> EASY
HARD <-------------------------------------------------------------------------------------------/

The key property of a hash chain is that it is EASY to compute the next value in the chain, given the previous value, but it is computationally infeasible (HARD) to compute the previous value given the current value. 
```

The chain starts by applying the hash function ($H$) to the seed value ($s$), resulting in the digest value ($d0$). This digest value ($d0$) is then used as input for the hash function again, producing the next digest value ($d1$). This process is repeated for each subsequent value in the chain.

In [4]:
# helper to get seed
def getSeed():
    return secrets.token_hex()

getSeed()

'd9e0ec3398af19ee8bb47c097cc824fc402638bf1e17a2aac2d78dc39efdb722'

### Implement Hash Chain

In [5]:
def createHashChain(n, digest):
    # start chain
    commitment = digest
    hash_chain = []
    # update chain
    for i in range(n):
        commitment = sha256(commitment.encode(__ENCODING__)).hexdigest()
        hash_chain.append(commitment)

    return hash_chain


def getHashChain(n, seed = None):
    if seed is None:
        seed = getSeed()

    # create hash chain
    hash_chain = createHashChain(n, seed)

    # chain in reversed order
    hash_chain.reverse()

    return hash_chain, seed

The chain is created as:

In [6]:
n = 5
createHashChain(n, getSeed())

['b69e7c47d72f7b824e1aa54416143631605bde4fcb8d4a54ac698f4b813f79bf',
 '304059391f9450f8b33af416f17beae0e7b91bdd729179c772ff5496c48c6425',
 'bf3da4ee90bfd5a5ef85bd8a5c7f02625308be4a25362b989d481e5a928070dc',
 '911a7e48c69ad810105257df690c2c9c5698ad5a9fdec4d70cbf23df88b06a6c',
 'd7c0bb4fdf4697b8ab47a6f9bc3fc356146a969f3e4e214a419212cfb3b5dc3a']

Final hash chain is reversed to get the properties that $H_0(seed)$ can be revealed. It will therefore be easy to calculate $H_{-1}$ but hard to find out $H_1$.

In [7]:
hash_chain, seed = getHashChain(n)
hash_chain

['5441f357cec52f230fcfd161713894e80cb144fa6bd04944b5bc60e155892a6d',
 'd600e607b650eccfde90f285518cdf10fa6584286a2cb418310503c336da2c0a',
 'd2db0966db1aa1882749d3e43f8be9751e4a58bbcc9e4765709ec2bc126aba8c',
 'e107119b9234558dc62a33156a110dc2294b6d409e0025cba96c97016f99ec62',
 'd9c3bb16d3c050d897bb0e55936c49bf14b0967e61af73696ff8c80d0270e8ca']

Show the properties of the chain, that given a hash the previous hash can easily be revealed.
Theirfor if $H_2$ is revealed $H_1$ can easily be computed.

In [8]:
hash_two = hash_chain[2]
print(hash_two)
# easy to hash input
hash_previous = sha256(hash_two.encode(__ENCODING__)).hexdigest()
# result hash
print(hash_previous)
# The property of hash chain determine that the hash is equal the previous hash in chain
hash_one = hash_chain[1]
print(hash_previous == hash_one)


d2db0966db1aa1882749d3e43f8be9751e4a58bbcc9e4765709ec2bc126aba8c
d600e607b650eccfde90f285518cdf10fa6584286a2cb418310503c336da2c0a
True


### Reproduce a hash chain

If you want to reproduce a hash chain, you need to know the starting value (also known as the seed value) and the hash function used to generate the chain. Once you have the starting value, you can compute the first hash value in the chain by applying the hash function to the starting value. Then, you can compute the second hash value by applying the hash function to the first hash value, and so on, until you have generated as many hash values as you need.

The hash function used to generate the chain should be a cryptographically secure hash function that is resistant to preimage attacks, meaning that it is computationally infeasible to find an input message that produces a given hash value, for example SHA-256.

It's important to note that if the seed value or the hash function used to generate the chain is compromised or weakened, the security of the hash chain may be compromised as well. Therefore, it's crucial to choose a strong seed value and a secure hash function, and to protect them from unauthorized access or tampering.

### How inequality-test work
#### GEQ
This illustrstion will demonstrate a hash chain as a rope instead.
Were lenght of Proof rope is the value to prove.

```
Proof:      |<--    FALSE   -->|<--  TRUE  -->
Proof Rope: -------------------|
```

Perform GEQ:
```
Proof Rope: -------------------|
Short Rope: ---------------    |        >> FALSE
Long Rope:  -------------------|-----   >> TRUE
```

The Short rope is not long enough but Long Rope is!

The Long Rope is therefore greater than the Proof Rope.

#### LEQ
With the same example but now we create a Base Rope:
```
Base Rope:  ------------------------------
```

Add padding after the lines to the length of Base Rope.
```
Base Rope:  ------------------------------
Proof Rope: -------------------ooooooooooo
Short Rope: ---------------ooooooooooooooo   
Long Rope:  ------------------------oooooo
```
Convert the leq-test to geq-test.
```
Proof Rope: ooooooooooo|
Short Rope: ooooooooooo|oooo   >> TRUE  
Long Rope:  oooooo     |       >> FALSE
```
The short Rope is greater than the Proof Rope!

### GEQ-test 
The inequality test with hash chain can be used because of the properties how a hash chain is reproduced.

To perform an inequality test on a hash chain, you would take two consecutive hash values from the chain, such as $H_i$ and $H_{i+1}$, and then try to find a value X such that $H_i$ = $H(X)$ using the hash function used to generate the chain. If the hash function is secure, it should be computationally infeasible to find such a value X, because the hash function is designed to be one-way.

In [9]:
# number to prove geq
# hash_zero public hash
# proof_digest_n hash used to prove it's larger than n
def geqHashProof(n, hash_zero, proof_digest_n):
    # sanity check
    if (len(proof_digest_n) != 64) or \
        (not isinstance(n, int)) or \
        (n < 0) or \
        (not isinstance(hash_zero, str)) or \
        (not isinstance(proof_digest_n, str)):
        return False

    # is it a start proof
    if n == 0 and proof_digest_n == hash_zero:
        return True
    else:
        # can hash zero be reproduced for a given range
        # were last value should be equal hash_zero 
        proof_chain_digest = createHashChain(n, proof_digest_n)[-1]
        return (proof_chain_digest == hash_zero)

Test how it's used

In [10]:
# user properties
age = 23
hash_chain = getHashChain(age)[0]
hash_zero = hash_chain[0]

# geq test
geq = 18
proof_digest_age = hash_chain[geq]
isGEQ = geqHashProof(geq, hash_zero, proof_digest_age)

print(age > geq)
print(isGEQ)

True
True


In [11]:
%%timeit
geqHashProof(geq, hash_zero, proof_digest_age)

45.3 µs ± 4.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Homework
For equality the length of chain need to be of modified, test!

### LEQ-test 
This inequality test is reversed, so instead the properties of an GEQ-test is used:

- N is much larger than threshold $t$ and the target value $x$
- If $t ≥ x$, then $N-t ≤ N-x$
- Make a hash chain for $N-x$


In [12]:
# properties
N = 100
age = 10
leq = 18

# diff
ageDiff = N - age
leqDiff = N - leq

# if age < leq then ageDiff > leqDiff
print(age <= leq)
print(ageDiff >= leqDiff)

# create
hash_chain = getHashChain(ageDiff + 1)[0]
hash_zero = hash_chain[0]

# hash digest user send to verifier
proof_digest_leq = hash_chain[leqDiff]
# leq is turned to a geq
isGEQ = geqHashProof(leqDiff, hash_zero, proof_digest_leq)
print(isGEQ)

True
True
True


### The Minimum Dominating Partition

A number $x$ dominates another number $y$ if each digit $x_i\geq y_i$. For instance, using base 10, 2345 dominates 1234. But 5011 does not dominate 4999. 
Let $MDP_b(x)$ denote the base-b minimum dominating partion of $x$ such that $x$ is split into parts that dominate other numbers. Examples:

* $MDP_{10}(84)=\{84,79\}$
* $MDP_{10}(113)=\{113,109,99\}$
* $MDP_{10}(999)=\{999\}$
* $MDP_{10}(1000)=\{1000,999\}$
* $MDP_{10}(3413)=\{3413,3409,3399,2999\}$

Then, based on the requested number to prove, Alice would pick one of these commitments: the one that has a long enough hash multichain to encode the number in question. If you understood the logic of hash multichain splitting, then you can easily realize that the commitment to:

From this logic, any number that can be proved is:

* 2999 can prove any number up to 2999
* 3399 can prove any number from 3000 to 3399
* 3409 can prove any number from 3400 to 3409
* 3413 can prove any number from 3410 to 3413

The number of 3399 could be used to prove the number 2500 as well, because its hashchains are long enough for the requirements of 2500 but we always pick the closest MDP.

In [13]:
def MDP(x):
    # lenX = len(str(x))
    lenX = math.ceil(math.log10(x+1))

    # init
    mdpVector = [x]

    # we can be lucky
    xp = (x+1) % (10 ** (lenX-1))
    if not xp:
        # return vector if it's zero
        return mdpVector

    # save minimum value
    minx = (x - xp)

    for i in range(1, lenX-1):
        xp = (x+1) % (10 ** i)
        diff = x - xp
        if diff not in mdpVector:
            if diff != minx:
                mdpVector.append(diff)
            else:
                break

    # add minimum value to the end
    mdpVector.append(minx)
    return mdpVector

In [14]:
print(MDP(84))
print(MDP(113))
print(MDP(1000))
print(MDP(3413))
print(MDP(3999))
print(MDP(11001))

[84, 79]
[113, 109, 99]
[1000, 999]
[3413, 3409, 3399, 2999]
[3999]
[11001, 10999, 9999]


In [15]:
%%timeit
MDP(6575)

3.01 µs ± 652 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Hash multichains

Hash multichains is an hash chain per digit position.

For example, an 8 digit number will require 8 hash chains.

In [16]:
def getMultiHashChain(value):
    # properties
    lenN = math.ceil(math.log10(value+1))

    seed = []
    multi_hash_chain = {}
    for i in range(lenN):
        seed.append(getSeed())
        multi_hash_chain[i] = getHashChain(10, seed[i])[0]

    return multi_hash_chain, seed

Create hash multichain with three digits

In [17]:
mdpValue = 115
multi_hash_chain = getMultiHashChain(mdpValue)[0]
multi_hash_chain

{0: ['0fde56cd5fec8d10ffcb0d9e490a960ad3621ae447c9197fd8a8d974fcadf05d',
  '3b957bf8913f408497f83a7171f3cecef1882422c6db9ca2453c733301c284c2',
  '5574c4b7c96ad7b97325e336285e81c79db99e6a14a9a4491b2398096feba26f',
  '9c8ce22956814bf51c25493dac4c0783f42555a06805d41d91753da44f708523',
  '43e474446d728a3f666830bd104a00bef53016de0c0aa94c6fd68a2a75b83d90',
  '6e5fadaa0695cc58692f4d4b07a4a6451950daa25767ca55c1b52f88f9128faf',
  '8a642af27d224644a0c8e1e28b588614eec10aefbcf4b7358175fd2ccae9ee67',
  'e9aa00cafd70689312b40440c25e54c4fd5f299bd319d75639813c59db208507',
  '71c8564e6d9b516bd01f429ed0d6f7c114565467636d343a0494cc4e2cc968ca',
  'dcdb263fa1a540f8d2d5df2def18b2be23253cda263ca4aa81c728af9fe7e468'],
 1: ['88cf77d6d1734da8418c0b976dc474550722088c96ff1bfba7750986a63e172e',
  '1f1d88618b350b68c065b44bf11d713d2a98def927ca4593d8285f74f263ae9c',
  '31237053097df78e482aa1983699eeb2d241bd463ae3c707f28357ccbb6b8ba2',
  '91a6eab12a3587dce381a04fa67a683f6599628325381f23266097ee5a676fdd',
  'cf018a50f9

### HashWires

HashWires rely on hash multichains. In hashwire a digit represent a value from the position in the hash multichain. The hash multichain has a hash chain per digitposition, each of length 10. For example, an 8 digit number will require a hash multichain with 8 hash chains when transforming digit number with hashwire.

If the same digit appears in the same position across multiple partitions, only one commitment to that digit needs to be generated and can be reused.

For example, an 8 digit number will never require more than 80 SHA26 operations!

HashWires are theirfor extremely efficent! 

In [18]:
def hashWireCommitmentGenerator(mdp_value, multi_hash_chain):
    strValue = str(mdp_value)
    lenV = len(strValue)
    maxN = len(multi_hash_chain)
    diff = maxN - lenV

    commitmentVector = []
    for i in range(lenV):
        digit = int(strValue[i])
        commitmentVector.append(multi_hash_chain[i + diff][digit])
    return commitmentVector


def getHashWireCommitments(mdp_vector, multi_hash_chain):
    return {value: hashWireCommitmentGenerator(value, multi_hash_chain) for value in mdp_vector}


Hash wire gives the digitwise commitments:

In [19]:
val=113
mdpVector = MDP(val)
multi_hash_chain = getMultiHashChain(val)[0]
hash_wire_claim = getHashWireCommitments(mdpVector, multi_hash_chain)
hash_wire_claim

{113: ['c73769ed8e77a3f8a07f916c946fdeebcc5bf41b4d198a457d118cbc268d0c65',
  '118c526c2d58f0d3f51c53943ab30e4154d65f74d253eb535fd83c140568eea5',
  '058cd98b8d0cd17672fa6fcf1260a863615b98c011c73177e3f8e5e77572fed8'],
 109: ['c73769ed8e77a3f8a07f916c946fdeebcc5bf41b4d198a457d118cbc268d0c65',
  'e0a21db2ab4d7e4c564af114bef5f8e17231a4104d924db00bba60778cbcec83',
  '892045155c56d79ecf301a6574979982034b8f8763a2d9941e6252a30b53a666'],
 99: ['395ad403e29ac4bf661a20134c17c8b93e7070310cc70b6cf2e87e9077c07a0c',
  '892045155c56d79ecf301a6574979982034b8f8763a2d9941e6252a30b53a666']}

In [20]:
# pretty plot hash wire with pandas so values can be mapped
pd.DataFrame(multi_hash_chain)

,0,1,2
0,3cdc181d029eca19deeb698a337db5efefa5db16578691...,e0a21db2ab4d7e4c564af114bef5f8e17231a4104d924d...,af2cde9a3ede65ff8cfe83b8d4e348cca7e6458e13e9d7...
1,c73769ed8e77a3f8a07f916c946fdeebcc5bf41b4d198a...,118c526c2d58f0d3f51c53943ab30e4154d65f74d253eb...,bdd8fb304fdf3cd39098eb862bed79f2130f0c2f3dd56f...
2,cc4c3740d7ded16afc2a6e98a363e529437fcef038c1a6...,e2b610d21f2178ce6b0e1f6aac83f0c938b56bc43e730e...,6fda9702ccba74a7bc87abed587a85a66e61126416f8f2...
3,520349ba8cfd2f00021c1b9bbf95b0a71916601b6e572a...,1f845c972c8fbe90878646f8ba393c01e4251ccbc3d513...,058cd98b8d0cd17672fa6fcf1260a863615b98c011c731...
4,ebc3acd790aca1c9403c55239fda47cf3ae1a663b5b1fa...,9f774b5881be5a78e8788f56a7f62842c6864c2a2d7485...,8edd2df603ae359bb1b47789c6f195377652b6db11b0ea...
5,2c89f4207ff2c6107e4316a8bd2fb458d6a41c274000bc...,93b163646a01a8d04b6f43fdc5b7e1aef00cedd7608db0...,b863ed4dfd3a31d0fbe99813d62a500f72b4dc5e5887aa...
6,25255628cb7b356c4f486324d5efbacd45cf23f10c7966...,578faf432727104c5b3c9dc79ed251facd8a0fdc2312b6...,b10ee6517f0b9f9c43ec9d7b360d6f4a49397816fcf330...
7,35a7ac0307fda92f0e3f5bf2f8c897cd5ac198e3917ede...,32e00a51173053c9e4e8bb89ecaaae0be99c8fe22d4a8a...,afde4c1bd9b2ba8ce41ae842e74c67576e1d9772469077...
8,d81dd8c233a29c14ee0659368161656740d8a519b3c0bd...,1828c6bd6fc4f676910e24c04fcc0c9e1304c733ae19fc...,e3c632ac6b772841cb594e08a8a2a6638be179a42e5c5e...
9,5af62260dc6bf653c67059a7d3243a4ac3db66584cf10b...,395ad403e29ac4bf661a20134c17c8b93e7070310cc70b...,892045155c56d79ecf301a6574979982034b8f8763a2d9...


## Wrap everything up

It has been showed that hashwire is extremy effective to use with inequality test. Think about a test for a value of 1000, then you need to calculate 1000 hashes for a regular hash chain but an hash wire with hash multichain just require 40 calculetions of hashes.

The extreme effectivnes of hashwirecan be demonstrated if the value than is 2000, how many more hashes do we calculate?

For the regular hash chain it would require 1000 additional calculations requires 1000 calculations of hashes to test for a value of 1000 dollars. The hashwire, on the other hand, would still require only 40 calculations of hashes for both cases.



## Wrap up implement
Now we know every function, lets sum it up!

### MDP

In [21]:
## Calculates the MDP vector in days from a birth date
class ProofProcessor:
    def __init__(self, birth_date):
        self.birth_date = date.fromisoformat(birth_date)
        self.__age_in_days = self.__age_in_days()
        self.mdp_list_age_in_days = MDP(self.__age_in_days)
    
    def __age_in_days(self):
        delta = datetime.now(tz=ZoneInfo("Europe/Stockholm")).date() - self.birth_date
        return delta.days

For instance, if someones age in days is 13755, than the  MPD10(13755)=[13755,13749,13699,12999,9999] ,  would need to create a hash wire with five (5) hash chains of length 10. 

In [22]:
birth_date = "1985-09-09" # value from an authentic source

#prepare MDP vector
user_mdpProof = ProofProcessor(birth_date)

print(f"The age MDP given birthdate {birth_date} is: ", user_mdpProof.mdp_list_age_in_days)

The age MDP given birthdate 1985-09-09 is:  [13764, 13759, 13699, 12999, 9999]


### Hash multichain

In [23]:
user_age_in_days = user_mdpProof.mdp_list_age_in_days[0]
lenU = len(str(user_age_in_days))

print(f"The users age in days are: ", user_age_in_days)
print(f"Number of columns of hash chains that need to be created for the hash wire is: ", lenU)

The users age in days are:  13764
Number of columns of hash chains that need to be created for the hash wire is:  5


In [24]:
# Creates the hashchain and the seed dictionaries
class HashChains:
    def __init__(self):
        self.seed_vector = []
        self.hash_chains = {}

    def __generate_hash_chains(self, value):
        h, s = getMultiHashChain(value)
        self.seed_vector = s
        self.hash_chains = h

    def create_hash_chains(self, value):
        self.__generate_hash_chains(value)


In [25]:
# initate the hash multichain
user_hashProof = HashChains()
# create hash multichain for user, by creating all hash chain for each column
user_hashProof.create_hash_chains(user_age_in_days)

In [26]:
# check seed
user_hashProof.seed_vector

['35fd95cf1c39af97be91ccbfd7135a84e637ba121b7af42e27cb0ab9ab004470',
 '90252b0d699939e951c05cb49b42fd7f75b6e0494432fd4586a662a191b1b1a7',
 '5360264a4637f0d047f2222d34dadb80b93b93f2d0b4f2eff6d9f34b151a8a50',
 '8bd8c6f0ce89f7819c875a640a2bf40c47f0896a76f64fa1bafb9f8ded03d2b0',
 '42471c878917bc59b75ef6afdb197ad645ea6a4b98710fa10c730b161e6f293e']

In [27]:
# check that hash multichain length is OK
lenH = len(user_hashProof.hash_chains)
lenU == lenH

True

### Commitment
The commitments is created with hash wires.

In [28]:
# Creates the hash wire commitments
class CommitmentProcessor:
    def __init__(self, mdp_list, hashProof):
        self.__mdp = mdp_list
        self.__hashProof = hashProof
        self.commitments = getHashWireCommitments(mdp_list, hashProof.hash_chains)

In [29]:
print(f"MDP:", user_mdpProof.mdp_list_age_in_days)
# The hash multichain is used to create the HashWire claim
user_hashWireCommitment = CommitmentProcessor(user_mdpProof.mdp_list_age_in_days, user_hashProof)
print("\nThe digitwise commitments to the MDP are:")
user_hashWireCommitment.commitments

MDP: [13764, 13759, 13699, 12999, 9999]

The digitwise commitments to the MDP are:


{13764: ['eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a3790f908a36f5eb8f25f3',
  '1a363cbf37afa22bcd7b1f3e3aa76095c7ce155a26902e87bd650e74c61a7b95',
  '34e840d85bb036c320e9934c92fd9949dee3a8fee9db97ce532c380b7d9da5de',
  'c84de75d69586fadde7ad7d3791bb37ebe5b0d8ca03c266d6c8a98126aebe782',
  '0fd76f50363ff0817f0d536ac40f8b0fbdd6bf968e854ee88ffc5092231d5c53'],
 13759: ['eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a3790f908a36f5eb8f25f3',
  '1a363cbf37afa22bcd7b1f3e3aa76095c7ce155a26902e87bd650e74c61a7b95',
  '34e840d85bb036c320e9934c92fd9949dee3a8fee9db97ce532c380b7d9da5de',
  'e421eccfcc3c88a7850b96370fee780009e22c196f3e971df6dc1f21f2e06af6',
  'ec3cdc5e7cc6f568a255e3f48b22a2b9a357378bf7f828a19cea01b412570519'],
 13699: ['eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a3790f908a36f5eb8f25f3',
  '1a363cbf37afa22bcd7b1f3e3aa76095c7ce155a26902e87bd650e74c61a7b95',
  '853766a22c01c8be7dc4d9f9b0c1d43647c7cadd8df89f82219a5f34a01ec832',
  'db2f90864818ad7a003d5dbf57904f07bb31a69afcdcb84fcb0dc8b1eb4d7ed9

In [30]:
pd.DataFrame(user_hashProof.hash_chains)

,0,1,2,3,4
0,6a54dccc8d598c41821de9c10e4962343eb487415818bc...,034479151245bf2d6085ab0f9da177ec35949ab0412db2...,26b34fe92c2469632817d2460fa9edfbfe5cdc6d62f615...,7b56d8a1216b6df178b1abd4577416805abf3a1bce1774...,563a2a8a4465fe87b1c0273da7f304a8c4157a57c51a48...
1,eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a379...,43577dc606738c888cbf09f26e979edf1a98ec1645c404...,8cd019bf770cefe62ea3036dca9ebde669a8c295ea56fd...,bf06ecef29bab0588e5e887f8b8ec360f525ecfd4bc0aa...,ab13e5460ee04ffd1f277e6ceccd6ddd35386f37ae7f78...
2,af7fb9c7c2a0f26baaa71d6f33c72e265939f11d5a14c8...,ba706ffa439a3a34cf10e08014e2291f8124e994d39726...,29a13aa1316535278d2e4bd53a75f65736ec61968a16ca...,9916d99ff7cad5ba4a79dfeb174ee8dcb7589910abcbf8...,1e933955436d29772958059cacd6c14061246a1ba98d89...
3,bfc0d52ab2ed617a98a22efa86a156be477ccbea12da30...,1a363cbf37afa22bcd7b1f3e3aa76095c7ce155a26902e...,2578db144a5d264b6eaa9061cd6e009c243a777ca6136a...,6d42c0c540b7f5adc04ce78ee16a2845eccfaa58bf8a5d...,bb9a97711c6322ecc35dc72e32fad84e291263b1258904...
4,3e2e46244e1f84ce53af4c5332dbe9ae30bbf5d964dc05...,1374db03ca3b21c3248e64a47e293794d86c3a1928f1a8...,1e80982957dfc22f0ccfcbab2cdb85176d4247916f0718...,9dfe33d94b50326430c65371eb4ce932756d2cba350d3b...,0fd76f50363ff0817f0d536ac40f8b0fbdd6bf968e854e...
5,6b1d58a3a518d752db6be47d118cbcde7a6289ef42721a...,0f4185f9d417e59a7e85697f6a63a2948ed4b46a3b22db...,cdf46f23aaeb236fba26e72ceef29937e575b500bb55b6...,e421eccfcc3c88a7850b96370fee780009e22c196f3e97...,2c4e0304e991d96decc291eb7ae89016bef23d011caff8...
6,3b04232b249bb66483ea08bde51c4e340e55c5c32b0383...,ae849b1eb3617e6c6f03e376674b6ba39aaff161b8459b...,853766a22c01c8be7dc4d9f9b0c1d43647c7cadd8df89f...,c84de75d69586fadde7ad7d3791bb37ebe5b0d8ca03c26...,3a0fa6d922c4305f4a3865ef882b4d01de8887fbb440b8...
7,5ac18f5e4e5f3c2acdc3b05802d484f1bb79c82b4c5351...,8749b9f0cf32b91c4a3259cf8446ef95f345db56b0b776...,34e840d85bb036c320e9934c92fd9949dee3a8fee9db97...,cce2ecb5a1a92f115a40be0a46cd1441832ec3362530ff...,5fd5f61d82b956bad11055310767f98f2f38d1ac58a23f...
8,61f76436b6fe665d7542346b54c26b3821ccc0130e748a...,f817db0b57fcc5a996c123c7fbbbd8f415d8728cc28a3d...,cce2d581019090f61e06ca089bdc5515e2b8055324a256...,de852e5138bd8d75fb65aeef29ee91433a7a8639a17acd...,167c3f480bb0828dedeeb59756e4f2b39738a7d54e036d...
9,4bb7e1860df0425b220ec046557d89fe77064b3afa1ee3...,3cc2dcfcebb2670ac022a4ba8632c5fdd460a91dfd583a...,9157130e4c6f3c1d22de4d99dbbcbb07f9be9e42acc40d...,db2f90864818ad7a003d5dbf57904f07bb31a69afcdcb8...,ec3cdc5e7cc6f568a255e3f48b22a2b9a357378bf7f828...


### ProtectedClaims
The hash wire commitments is padded with decoy messages (misleading information used to confuse potential attackers).

In [31]:
# Creates the input to the _sd
class ProtectedClaims:
    def __init__(self, seeds, commitments):
        self.iat = datetime.now(tz=ZoneInfo("Europe/Stockholm")).date().isoformat()
        self.__seeds = seeds
        self.__commitments = commitments
        self.disclosure_inputs = None

        # set properties
        self.__disclosure_input()

    def __disclosure_input(self):
        maxN = 10
        # commitments is padded to length of maxN
        lenDecoys = maxN - len(self.__seeds)
        # create decoys
        # the decoy is here easily detected when it's only one element!
        decoys = [[sha256(secrets.token_bytes()).hexdigest()] for i in range(lenDecoys)]
        # create disclousre
        values = list(self.__commitments.values()) + decoys
        # randomize list
        random.shuffle(values)
        keys = list(range(len(values)))
        self.disclosure_inputs = dict(zip(keys, values))


In [32]:
# The claims contain decoys and are shuffled
user_attestation = ProtectedClaims(user_hashProof.seed_vector, user_hashWireCommitment.commitments)
print("The hash wire commitments and decoys are listed:")
user_attestation.disclosure_inputs

The hash wire commitments and decoys are listed:


{0: ['eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a3790f908a36f5eb8f25f3',
  'ba706ffa439a3a34cf10e08014e2291f8124e994d3972633c744ecbe33fc2536',
  '9157130e4c6f3c1d22de4d99dbbcbb07f9be9e42acc40daa2b67aa307f523616',
  'db2f90864818ad7a003d5dbf57904f07bb31a69afcdcb84fcb0dc8b1eb4d7ed9',
  'ec3cdc5e7cc6f568a255e3f48b22a2b9a357378bf7f828a19cea01b412570519'],
 1: ['f5ea346745a8b27e21f526fab11f07f5a1fbac8bea970081484820d771d650bf'],
 2: ['eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a3790f908a36f5eb8f25f3',
  '1a363cbf37afa22bcd7b1f3e3aa76095c7ce155a26902e87bd650e74c61a7b95',
  '34e840d85bb036c320e9934c92fd9949dee3a8fee9db97ce532c380b7d9da5de',
  'e421eccfcc3c88a7850b96370fee780009e22c196f3e971df6dc1f21f2e06af6',
  'ec3cdc5e7cc6f568a255e3f48b22a2b9a357378bf7f828a19cea01b412570519'],
 3: ['f0f88f2e72a3b25aa66d29fffca3b05b9075904851515f77fcbdfb526ad8a1ae'],
 4: ['e4210b3060bf7b30b3f181d0711ba1d14a737d91f9671b493b7283e3d67ab987'],
 5: ['eb45d6d5cdacec6ae42c88df94cf963bdcc2389995a3790f908a36f5eb8f25f3

## Reference
- https://zkproof.org/2021/05/05/hashwires-range-proofs-from-hash-functions/
- https://eprint.iacr.org/2021/297
- https://research.facebook.com/publications/hashwires-hyperefficient-credential-based-range-proofs/